<a href="https://colab.research.google.com/github/seplanas/Regresion-logistica/blob/main/Actividad_7_A01702733.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv('/content/cuentas_credicel.csv',
                 encoding = 'latin')

<ipython-input-2-71e76e9bd8d6>:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/cuentas_credicel.csv',


In [ ]:
df.info()

In [ ]:
df.head(5)

In [ ]:
df['limite_credito'] = df['limite_credito'].str.extract('(\d+\.\d+|\d+)').astype(float)

In [ ]:
print(df['limite_credito'])

0        5000.0
1        6000.0
2           0.0
3        7000.0
4        7000.0
          ...  
22730       NaN
22731       NaN
22732       NaN
22733       NaN
22734       NaN
Name: limite_credito, Length: 22735, dtype: float64


**Limpieza de valores nulos**

In [ ]:
#Rellenamos nulos
df =df.fillna(method="bfill")
df =df.fillna(method="ffill")

In [ ]:
valores_nulos=df.isnull().sum()
print(valores_nulos)

**Limpieza de outliers**

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

<ipython-input-8-a3b1a3480c80>:1: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q1 = df.quantile(0.25)
<ipython-input-8-a3b1a3480c80>:2: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q3 = df.quantile(0.75)
<ipython-input-8-a3b1a3480c80>:4: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]


### Valores de X y riesgo como la variable Y a comparar

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score

# Cargar tus datos (asegúrate de que `df` contenga tus datos reales)
# df = pd.read_csv('tudataset.csv')

# Definir un umbral para clasificar como "alto riesgo" o "bajo riesgo"
umbral = 0.5  # Puedes ajustar este umbral según tus necesidades

# Crear una nueva variable categórica basada en el umbral
df['riesgo_binario'] = df['riesgo'].apply(lambda x: 1 if x > umbral else 0)

# Seleccionar solo las variables numéricas para el análisis, excluyendo "riesgo" y "riesgo_binario"
numeric_cols = df.select_dtypes(include='number').drop(['riesgo', 'riesgo_binario'], axis=1).columns

# Crear una lista para almacenar los resultados de los 10 modelos
results = []

# Realizar 10 iteraciones para ajustar modelos con variables numéricas aleatorias
for i in range(10):
    # Seleccionar un subconjunto aleatorio de variables numéricas
    random_numeric_cols = np.random.choice(numeric_cols, size=3, replace=False)  # Aquí se seleccionan 3 variables, puedes ajustar esto

    # Seleccionar las variables independientes (X) y la variable objetivo (y)
    X = df[random_numeric_cols]
    y = df['riesgo_binario']  # Utilizamos la variable binaria 'riesgo_binario' como objetivo

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Inicializar el modelo de regresión logística
    model = LogisticRegression()

    # Ajustar el modelo a los datos de entrenamiento
    model.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred = model.predict(X_test)

    # Matriz de confusión
    matriz = confusion_matrix(y_test, y_pred)
    print('Modelo ', i + 1)
    print('Matriz de Confusión:')
    print(matriz)
    print()

    # Calcular las métricas de rendimiento
    precision = precision_score(y_test, y_pred, average="binary")
    print('Precisión del modelo', i + 1)
    print(precision)
    print()

    exactitud = accuracy_score(y_test, y_pred)
    print('Exactitud del modelo', i + 1)
    print(exactitud)
    print()

    sensibilidad = recall_score(y_test, y_pred, average="binary")
    print('Sensibilidad del modelo', i + 1)
    print(sensibilidad)
    print('*' * 30)

    # Guardar los resultados en la lista
    results.append({
        'Modelo': i + 1,
        'Variables utilizadas': ', '.join(random_numeric_cols),
        'Precisión': precision,
        'Exactitud': exactitud,
        'Sensibilidad': sensibilidad
    })

# Crear un DataFrame con los resultados
results_df = pd.DataFrame(results)

# Imprimir los resultados
print(results_df)


In [ ]:
results_df

,Modelo,Variables utilizadas,Precisión,Exactitud,Sensibilidad
0,1,"folio_solicitud, limite_credito, pagos_realizados",0.763210,0.747061,0.935247
1,2,"edad_cliente, porc_eng, enganche",0.805986,0.803657,0.954184
2,3,"descuento, costo_total, folio_solicitud",0.713848,0.712233,0.995113
3,4,"porc_eng, precio, limite_credito",0.823810,0.820200,0.951130
4,5,"limite_credito, enganche, pagos_realizados",0.815400,0.825860,0.976787
5,6,"monto_accesorios, porc_eng, monto_financiado",0.803911,0.801480,0.954184
6,7,"semana, puntos, porc_eng",0.829728,0.804963,0.913867
7,8,"folio, semana_actual, inversion",0.712669,0.712669,1.000000
8,9,"folio_solicitud, porc_eng, semana_actual",0.787481,0.783195,0.952963
9,10,"pagos_realizados, semana_actual, fraude",0.712669,0.712669,1.000000


###Transformación de variables dicotómicas

In [ ]:
for column in df.columns:
    unique_values = np.unique(df[column])
    num_unique_values = len(unique_values)
    if num_unique_values <= 4:
        print(column, 'tiene', num_unique_values, 'variable(s) única(s)')

plazo tiene 4 variable(s) única(s)
monto_accesorios tiene 1 variable(s) única(s)
status tiene 1 variable(s) única(s)
fraude tiene 1 variable(s) única(s)
inversion tiene 1 variable(s) única(s)
reautorizacion tiene 1 variable(s) única(s)
status_cuenta tiene 4 variable(s) única(s)
riesgo_binario tiene 2 variable(s) única(s)


En este caso solo consideraría prudente transformar plazo y status_cuenta dado que no tienen muchas variables. Las demas variables que cuentan con único valor las eliminaría del modelo dado que no aportan valor al mismo. Esto en un futuro caso de tener que usar el modelo. Para esta entrega no está dentro del alcance del proyecto.

In [ ]:
unico1 = np.unique(df['plazo'])
print(unico1)
unico2 = np.unique(df['status_cuenta'])
print(unico2)

['13S' '26S' '39S' '52S']
['Atraso' 'Cancelado' 'Corriente' 'Mora']


In [ ]:
df['status_cuenta']= df['status_cuenta'].replace(["Atraso",
                                                  "Cancelado",
                                                  "Mora"], "No Corriente")
df['plazo'] = df['plazo'].replace(["13S",
                                   "26S",
                                   "39S"], "Menos de un año")
df[['status_cuenta','plazo']]

,status_cuenta,plazo
3296,No Corriente,Menos de un año
3413,No Corriente,Menos de un año
3493,Corriente,Menos de un año
3657,No Corriente,Menos de un año
3664,No Corriente,Menos de un año
...,...,...
22727,No Corriente,Menos de un año
22728,No Corriente,Menos de un año
22730,No Corriente,Menos de un año
22731,No Corriente,Menos de un año
